In [1]:
import json
import pandas as pd

train = pd.read_csv('./datasets/train.csv')
paper_train_folder = './datasets/train'
# print(train.shape)
# print(train[train['pub_title'] == 'Risk factors and global cognitive status related to brain arteriolosclerosis in elderly individuals'])

train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

# print(train.head())
# print(train.shape)

# train['dataset_title'].str.find('|')
train.iloc[3]['pub_title']

'Risk factors and global cognitive status related to brain arteriolosclerosis in elderly individuals'

In [2]:
papers = {}
for paper_id in train['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

In [3]:
papers[train['Id'][0]]

[{'section_title': 'Abstract',
  'text': "The aim of this study was to identify if acquiring ICT skills through DOT Lebanon's ICT training program (a local NGO) improved income generation opportunities after 3-months of completing the training. The target population was the NGO's vulnerable young beneficiaries. This study was completed in an effort to find creative and digital solutions to the high rate of youth unemployment in Lebanon (37%), one of the highest rates in the world. Results showed that 48% of beneficiaries who were unemployed at baseline, were exposed to at least one income generation opportunity 3 months after completing the DOT Lebanon training. Also, 49% of beneficiaries who were already employed at baseline were exposed to at least one income generation opportunity. Gender, English proficiency and governorate were variables that were found to be statistically significant. Males were more likely than females to be exposed to income generation opportunities. Those who 

In [4]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))


In [5]:
# Hyperparameters
MAX_LENGTH = 64 # max no. words for each sentence.
OVERLAP = 20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

In [6]:
import re
from tqdm import tqdm
import random

cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
ner_data = []

pbar = tqdm(total=len(train))
for i, id, dataset_label in train[['Id', 'dataset_label']].itertuples():
    # paper
    paper = papers[id]
    
    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]
    
    # sentences
    sentences = set([clean_training_text(sentence) for section in paper 
                 for sentence in section['text'].split('.') 
                ])
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos += 1
            ner_data.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']): 
            ner_data.append(tags)
            cnt_neg += 1
    
    # process bar
    pbar.update(1)
    pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

# shuffling
random.shuffle(ner_data)


Training data size: 47202 positives + 514263 negatives: 100%|████████████████████████████████████████████████████████████████████████████| 14316/14316 [01:12<00:00, 198.04it/s]

In [7]:
gazetteer = train['cleaned_label'].unique()
gazetteer

array(['program for the international assessment of adult competencies',
       'trends in international mathematics and science study',
       'agricultural resources management survey',
       'adni|alzheimer s disease neuroimaging initiative adni ',
       'genome sequence of covid 19',
       'early childhood longitudinal study',
       'baltimore longitudinal study of aging blsa |baltimore longitudinal study of aging',
       'noaa tide gauge', 'baltimore longitudinal study of aging', 'adni',
       'optimum interpolation sea surface temperature|noaa optimum interpolation sea surface temperature',
       'beginning postsecondary students', 'slosh model',
       'covid 19 death data',
       'international best track archive for climate stewardship',
       'national education longitudinal study|early childhood longitudinal study|education longitudinal study',
       'survey of earned doctorates',
       'education longitudinal study|national education longitudinal study',
       '

In [8]:
import torch
from transformers import BertTokenizerFast, BertModel

# Load pre-trained BERT model and fast tokenizer
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Word shape and short word shape helper functions
def word_shape(word):
    shape = ''
    for char in word:
        if char.isupper():
            shape += 'X'
        elif char.islower():
            shape += 'x'
        elif char.isdigit():
            shape += 'd'
        else:
            shape += char
    return shape

def short_word_shape(word):
    # Simplified word shape: uppercase letters (X), lowercase letters (x), digits (d)
    if word.isupper():
        return 'X'
    elif word.islower():
        return 'x'
    elif word.isdigit():
        return 'd'
    else:
        return 'other'

# Check if word exists in the gazetteer
def in_gazetteer(word):
    return any(word in gaz.lower() for gaz in gazetteer)


# Prefix and Suffix helpers
def prefixes_suffixes(word, length=4):
    return {
        'prefix': word[:length] if len(word) >= length else word,
        'suffix': word[-length:] if len(word) >= length else word,
    }

# Example usage: Extract embeddings for a word in a sentence
def get_word_embedding(sentence, word_index):
    # Tokenize the sentence with padding, truncation, and word-level mapping
    inputs = bert_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, is_split_into_words=True)

    # Get the hidden states from BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)

    hidden_states = outputs.last_hidden_state

    # Map the word index to the tokenized output using the `word_ids()` method
    word_ids = inputs.word_ids()  # Returns the mapping between original words and tokenized indices

    # Find the corresponding token index for the word at `word_index`
    token_indices = [i for i, word_id in enumerate(word_ids) if word_id == word_index]
    
    if len(token_indices) == 0:
        raise IndexError(f"No tokens found for word index {word_index}")

    # Average the embeddings for subword tokens if there are multiple subword tokens for the word
    embedding = torch.mean(hidden_states[0, token_indices], dim=0).numpy()

    return embedding

# Part-of-Speech tagging (simplified for demonstration purposes)
def get_pos(word):
    if word[0].isupper():
        return 'NOUN'
    elif word in ['the', 'is', 'a', 'an', 'on']:
        return 'DET'
    else:
        return 'OTHER'

# Main feature extraction function
def extract_features(sentence, i):
    """
    Extract features for word `i` in the sentence.
    """
    word = sentence[i]
    
    # Base features: word identity, word shape, short shape, POS, embeddings
    features = {
        'word': word,
        'is_first': i == 0,
        'is_last': i == len(sentence) - 1,
        'is_capitalized': word[0].isupper(),
        'is_all_caps': word.isupper(),
        'is_all_lower': word.islower(),
        'prefix': prefixes_suffixes(word)['prefix'],
        'suffix': prefixes_suffixes(word)['suffix'],
        'word_shape': word_shape(word),
        'short_word_shape': short_word_shape(word),
        'pos': get_pos(word),
        'embedding': get_word_embedding(sentence, i),
    }

    # Adding features for neighboring words (previous and next)
    if i > 0:
        word_prev = sentence[i-1]
        features.update({
            'prev_word': word_prev,
            'prev_word_shape': word_shape(word_prev),
            'prev_short_word_shape': short_word_shape(word_prev),
            'prev_pos': get_pos(word_prev),
            'prev_embedding': get_word_embedding(sentence, i-1),
        })
    else:
        features.update({
            'prev_word': '<START>',
            'prev_word_shape': '<START>',
            'prev_short_word_shape': '<START>',
            'prev_pos': '<START>',
            'prev_embedding': torch.zeros(bert_model.config.hidden_size).numpy(),
        })
    
    if i < len(sentence) - 1:
        word_next = sentence[i+1]
        features.update({
            'next_word': word_next,
            'next_word_shape': word_shape(word_next),
            'next_short_word_shape': short_word_shape(word_next),
            'next_pos': get_pos(word_next),
            'next_embedding': get_word_embedding(sentence, i + 1),
        })
    else:
        features.update({
            'next_word': '<END>',
            'next_word_shape': '<END>',
            'next_short_word_shape': '<END>',
            'next_pos': '<END>',
            'next_embedding': torch.zeros(bert_model.config.hidden_size).numpy(),
        })
    
    return features

def sent2features(sent):
    return [extract_features(sent, i) for i in range(len(sent))]

# Example usage
sentence = ["The", "National", "Education", "Longitudinal", "Study", "was", "used"]
features = sent2features(sentence)


/Users/juancarlosroldan/Projects/ai_nlp/crfner_model/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training data size: 47202 positives + 514263 negatives: 100%|████████████████████████████████████████████████████████████████████████████| 14316/14316 [01:23<00:00, 198.04it/s]/Users/juancarlosroldan/Projects/ai_nlp/crfner_model/nlp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Extract tokens (words) from the data
tokenized_sentences = [[word for word, label in sentence] for sentence in ner_data][:100]

# Feature extraction with a progress bar using tqdm
features = []
for sentence in tqdm(tokenized_sentences, desc="Extracting features", position=0, leave=True):
    features.append(sent2features(sentence))

# Example of showing the features for the first sentence
for token_features in features[0]:
    print(token_features)


Extracting features: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:16<00:00,  3.17s/it]

{'word': 'Therefore', 'is_first': True, 'is_last': False, 'is_capitalized': True, 'is_all_caps': False, 'is_all_lower': False, 'prefix': 'Ther', 'suffix': 'fore', 'word_shape': 'Xxxxxxxxx', 'short_word_shape': 'other', 'pos': 'NOUN', 'embedding': array([-1.28157318e-01,  4.00104910e-01, -2.33828984e-02, -3.68920833e-01,
       -1.66671395e-01, -2.65139610e-01,  1.94983613e-02, -1.40159488e-01,
        2.40642756e-01, -2.87367910e-01, -4.53022361e-01,  3.04534920e-02,
       -2.47156247e-01,  6.84946850e-02,  2.42010772e-01,  2.87392676e-01,
        4.14857745e-01,  8.00869688e-02, -2.45083377e-01, -2.98344847e-02,
       -6.07500933e-02,  5.36851168e-01, -5.57639673e-02,  9.64184463e-01,
        6.25720918e-01, -2.87650257e-01,  1.61639988e-01, -4.15060222e-01,
       -6.32159039e-02, -1.82722151e-01,  4.94559348e-01, -5.21672741e-02,
       -1.87345624e-01, -1.56652927e-01,  5.74831843e-01,  4.66635227e-01,
       -1.69727445e-01,  6.81272149e-01,  9.29946080e-04,  2.16610268e-01,
   

In [10]:
print(len(features))
print(len(ner_data))

100
561465


In [11]:
# Commented out because this is this is an implementation from
# sklearn_crfsuite

# import sklearn_crfsuite
# from sklearn_crfsuite import metrics
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# import logging
# import time

# # Custom logger for CRF training to integrate with tqdm
# class TqdmLoggingHandler(logging.Handler):
#     def __init__(self, tqdm_bar):
#         super().__init__()
#         self.tqdm_bar = tqdm_bar

#     def emit(self, record):
#         self.tqdm_bar.update(1)

# # Assuming your features array is already extracted and stored in 'features'

# start = time.time()
# # Extract the labels (IOB tags) from your original data
# print('setting labels')
# labels = [[label for word, label in sentence] for sentence in ner_data]
# end = time.time()

# print(f'Label setting time: {end - start}')
# # Split the data into training and test sets (e.g., 80% train, 20% test)
# start = time.time()
# print('train-test splitting')
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
# end = time.time()
# print(f'Splitting time: {end - start}')

# # Initialize the CRF model with a custom logging handler
# crf = sklearn_crfsuite.CRF(
#     algorithm='lbfgs',
#     c1=0.1,
#     c2=0.1,
#     max_iterations=100,  # Define the max iterations
#     all_possible_transitions=True
# )

# def tqdm_callback(trainer, tqdm_bar):
#     def _callback(log):
#         tqdm_bar.update(1)
#     trainer.on_iteration = _callback

# print('Start of CRF Training')
# start = time.time()
# # Create a tqdm progress bar with total iterations equal to crf.max_iterations
# with tqdm(total=crf.max_iterations, desc="Training CRF") as pbar:
    
#     # Attach tqdm progress bar to the trainer's on_iteration callback
#     tqdm_callback(crf, pbar)

#     # Train the CRF model
#     crf.fit(X_train, y_train)
# end = time.time()
# print(f'Elapsed time: {end - start}')

# # Make predictions on the test set
# y_pred = crf.predict(X_test)
# print('finished prediction')

In [17]:
print(features[0][0].keys())

dict_keys(['word', 'is_first', 'is_last', 'is_capitalized', 'is_all_caps', 'is_all_lower', 'prefix', 'suffix', 'word_shape', 'short_word_shape', 'pos', 'embedding', 'prev_word', 'prev_word_shape', 'prev_short_word_shape', 'prev_pos', 'prev_embedding', 'next_word', 'next_word_shape', 'next_short_word_shape', 'next_pos', 'next_embedding'])


In [19]:
import numpy as np


def log_sum_exp(arr):
    """
    Perform the log-sum-exp trick for numerical stability.
    Given an array `arr`, compute log(sum(exp(arr))).
    """
    max_val = np.max(arr)
    return max_val + np.log(np.sum(np.exp(arr - max_val)))


class CRF:
    def __init__(self, labels):
        self.labels = labels  # List of possible labels (e.g., ["B", "I", "O"])
        self.num_labels = len(labels)
        self.weights = {}  # Dictionary to store feature weights, initialized randomly
    
    def initialize_weights(self):
        """
        Initialize weights for a predefined set of consistent feature types.
        """
        # Define the consistent feature keys
        feature_keys = [
            'word', 'is_first', 'is_last', 'is_capitalized', 'is_all_caps', 'is_all_lower',
            'prefix', 'suffix', 'word_shape', 'short_word_shape', 'pos', 'embedding',
            'prev_word', 'prev_word_shape', 'prev_short_word_shape', 'prev_pos', 'prev_embedding',
            'next_word', 'next_word_shape', 'next_short_word_shape', 'next_pos', 'next_embedding'
        ]
    
        # Initialize weights for each feature key
        for feature in feature_keys:
            if "embedding" in feature:
                # For embedding features (which are lists), initialize a weight for the sum of embeddings
                self.weights[feature + "_sum"] = np.random.uniform(-0.01, 0.01)
            else:
                # Initialize weight for other scalar features
                self.weights[feature] = np.random.uniform(-0.01, 0.01)

    def compute_transition_score(self, features):
        """
        Compute the transition score as the weighted sum of feature values.
        Handles numeric features and converts non-numeric features to numeric form.
        """
        score = 0
        for feature, value in features.items():
            if feature in self.weights:
                if isinstance(value, (int, float)):  # If the value is numeric
                    score += self.weights[feature] * value
                elif isinstance(value, list):  # Handle embeddings or other lists
                    # Example: Sum the embedding vector
                    score += self.weights[feature] * np.sum(value)
                else:
                    # For non-numeric values (e.g., words, POS tags), we can either ignore them
                    # or assign them a numeric value, e.g., 1 if the feature exists
                    score += self.weights[feature] * 1  # Assign a default value of 1 for non-numeric features
        return score

    def forward_pass(self, sentence):
        T = len(sentence)
        alpha = np.zeros((T, self.num_labels))
        
        # Initialization: start at position 0
        for j in range(self.num_labels):
            features = extract_features(sentence, 0)
            alpha[0, j] = self.compute_transition_score(features)
        
        # Forward recursion
        for t in range(1, T):
            for j in range(self.num_labels):
                alpha[t, j] = np.log(np.sum([np.exp(alpha[t-1, k] + self.compute_transition_score(extract_features(sentence, t))) for k in range(self.num_labels)]))
        
        return alpha

    def backward_pass(self, sentence):
        T = len(sentence)
        beta = np.zeros((T, self.num_labels))
        
        # Initialization: set beta at the last position
        beta[T-1, :] = 1
        
        # Backward recursion
        for t in range(T-2, -1, -1):
            for y_i in range(self.num_labels):
                beta[t, y_i] = np.sum([beta[t+1, y_next] * np.exp(self.compute_transition_score(extract_features(sentence, t+1))) for y_next in range(self.num_labels)])
        
        return beta

    def log_likelihood(self, sentence, true_labels):
        """
        Compute the log-likelihood of a given sequence of labels.
        """
        alpha = self.forward_pass(sentence)
        beta = self.backward_pass(sentence)
        
        # Compute the log-likelihood: sum of transition scores for the true labels
        likelihood = 0
        for t in range(len(sentence)):
            features = extract_features(sentence, t)
            likelihood += self.compute_transition_score(features)
        
        # Subtract the log of the partition function (computed as the sum of alpha and beta)
        Z = np.sum(np.exp(alpha[-1, :]))  # Partition function from the forward pass
        return likelihood - np.log(Z)

    def update_weights(self, sentence, true_labels, learning_rate=0.01):
        """
        Perform a gradient update of the feature weights using SGD with numerical stability.
        """
        # Forward-backward probabilities
        alpha = self.forward_pass(sentence)
        beta = self.backward_pass(sentence)
    
        # Calculate the partition function using log-sum-exp for stability
        Z = log_sum_exp(alpha[-1, :])  # Partition function Z(x)
    
        # Update weights using true labels
        for t in range(len(sentence)):
            features = extract_features(sentence, t)
            for feature, value in features.items():
                if feature in self.weights:
                    if isinstance(value, (int, float)):  # Numeric values
                        self.weights[feature] += learning_rate * value
                    elif isinstance(value, list):  # Handle embeddings or lists
                        self.weights[feature] += learning_rate * np.sum(value)
                    else:  # Handle non-numeric values (e.g., strings)
                        self.weights[feature] += learning_rate * 1
    
        # Calculate feature expectations under the model distribution
        for t in range(len(sentence)):
            # Compute the log probabilities for all labels at this time step
            log_marginals = alpha[t, :] + beta[t, :] - Z
            
            # Prevent overflow by normalizing the log-marginals before exponentiation
            max_log_marginals = np.max(log_marginals)
            marginals = np.exp(log_marginals - max_log_marginals)
    
            # Normalize the marginals to ensure they sum to 1
            marginals /= np.sum(marginals)
    
            for y in range(self.num_labels):
                features = extract_features(sentence, t)
                for feature, value in features.items():
                    if feature in self.weights:
                        if isinstance(value, (int, float)):
                            self.weights[feature] -= learning_rate * marginals[y] * value
                        elif isinstance(value, list):  # Embeddings or lists
                            self.weights[feature] -= learning_rate * marginals[y] * np.sum(value)
                        else:  # Non-numeric feature (e.g., strings)
                            self.weights[feature] -= learning_rate * marginals[y] * 1


    def train(self, sentences, labels, epochs=10, learning_rate=0.01):
        """
        Train the CRF model using stochastic gradient descent (SGD).
        """
        for epoch in range(epochs):
            for sentence, true_labels in zip(sentences, labels):
                self.update_weights(sentence, true_labels, learning_rate)
            print(f"Epoch {epoch + 1} completed.")
    
    def viterbi_decode(self, sentence):
        """
        Decode the most likely sequence of labels for a sentence using the Viterbi algorithm.
        """
        T = len(sentence)
        num_labels = self.num_labels
        
        # DP table and backpointers
        dp = np.zeros((T, num_labels))
        backpointers = np.zeros((T, num_labels), dtype=int)
        
        # Initialize DP table
        for j in range(num_labels):
            dp[0, j] = self.compute_transition_score(extract_features(sentence, 0))
        
        # Viterbi recursion
        for t in range(1, T):
            for j in range(num_labels):
                max_score, max_label = max(
                    (dp[t-1, k] + self.compute_transition_score(extract_features(sentence, t)), k)
                    for k in range(num_labels)
                )
                dp[t, j] = max_score
                backpointers[t, j] = max_label
        
        # Backtrace to get the best sequence
        best_sequence = []
        best_last_label = np.argmax(dp[T-1])
        best_sequence.append(self.labels[best_last_label])
        
        for t in range(T-2, -1, -1):
            best_last_label = backpointers[t+1, best_last_label]
            best_sequence.append(self.labels[best_last_label])
        
        return list(reversed(best_sequence))

# Example Usage
labels = ["B", "I", "O"]  # Labels for beginning, inside, outside of a citation
crf = CRF(labels)

# Sample sentences (tokens) and true labels
sentences = [["The", "National", "Education", "Longitudinal", "Study", "was", "used"],
             ["Data", "from", "the", "National", "Longitudinal", "Survey", "was", "analyzed"]]
true_labels = [["O", "B", "I", "I", "I", "O", "O"],
               ["O", "O", "O", "B", "I", "I", "O", "O"]]

crf.initialize_weights()

# Train the CRF
crf.train(sentences, true_labels, epochs=20, learning_rate=0.01)

# Test decoding with the Viterbi algorithm
sentence = ["The", "National", "Education", "Longitudinal", "Study", "was", "National", "Education", "Longitudinal"]
predicted_labels = crf.viterbi_decode(sentence)
print(f"Predicted labels: {predicted_labels}")


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
Epoch 6 completed.
Epoch 7 completed.
Epoch 8 completed.
Epoch 9 completed.
Epoch 10 completed.
Epoch 11 completed.
Epoch 12 completed.
Epoch 13 completed.
Epoch 14 completed.
Epoch 15 completed.
Epoch 16 completed.
Epoch 17 completed.
Epoch 18 completed.
Epoch 19 completed.
Epoch 20 completed.
Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']


In [20]:
def calculate_precision_recall(true_labels, predicted_labels, positive_labels=["B", "I"]):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true, pred in zip(true_labels, predicted_labels):
        if true in positive_labels and pred in positive_labels:
            true_positives += 1
        elif pred in positive_labels and true not in positive_labels:
            false_positives += 1
        elif true in positive_labels and pred not in positive_labels:
            false_negatives += 1

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

precision, recall = calculate_precision_recall(true_labels, predicted_labels, positive_labels=["B", "I"])
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

Precision: 0.00, Recall: 0.00
